![ga4](https://www.google-analytics.com/collect?v=2&tid=G-6VDTYWLKX6&cid=1&en=page_view&sid=1&dl=statmike%2Fvertex-ai-mlops%2FTips&dt=BigQuery+-+Python+Client.ipynb)

# Python BigQuery Client(s)

This notebook covers many ways of interacting with BigQuery tables from Python, including:
- ways unique to Jupyter notebooks like magics
- built in methods for pandas like pandas-gbq methods
- the BigQuery Python clinet
- BigQuery storage api for fast, multiprocessing, reads directly from BigQuery storage


**Notes:**
- The `LIMIT 5` statement does limit the number of rows returned by BigQuery to 5 but BigQuery still does a full table scan.  If you have a table larger than 1GB and want to limit the rows scanned for a quick review then replacing `LIMIT 5` with `TABLESAMPLE SYSTEM (1 PERCENT)` would be more efficient.  For tables under 1GB it will still return the full table.  More on [Table Sampling](https://cloud.google.com/bigquery/docs/table-sampling)
- Each of the examples below run the same query in BigQuery.  The query is cached on the first run for up to 24 hours.  This means the subsequent, identical queries will not scan the data and instead use the cached results table.  More information on [Using cached query results](https://cloud.google.com/bigquery/docs/cached-results).



**Resources:**
- [BigQuery Python Client](https://cloud.google.com/python/docs/reference/bigquery/latest)
    - Interact with BigQuery compute to run queries
- [BigQuery Storage API Python Client](https://cloud.google.com/python/docs/reference/bigquerystorage/latest)
    - directly read from BigQuery storage through streams which support multiprocessing
- Using BigQuery From Python, Notebooks in This Repository
    - [01 - Data Sources/01 - BigQuery - Table Data Sources](../01%20-%20Data%20Sources/01%20-%20BigQuery%20-%20Table%20Data%20Source.ipynb)
    - [03 - BigQuery ML (BQML)/03 - Introduction to BigQuery ML (BQML)](../03%20-%20BigQuery%20ML%20(BQML)/03%20-%20Introduction%20to%20BigQuery%20ML%20(BQML).ipynb)
    - [Applied Forecasting/BigQuery Time Series Forecasting Data Review and Preparation](../Applied%20Forecasting/BigQuery%20Time%20Series%20Forecasting%20Data%20Review%20and%20Preparation.ipynb)


---
## Colab Setup

To run this notebook in Colab click [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Tips/BigQuery%20-%20Python%20Client.ipynb) and run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    import google.colab
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [17]:
# tuples of (import name, install name)
packages = [
    ('google.cloud.bigquery', 'google-cloud-bigquery'),
    ('google.cloud.bigquery_storage', 'google-cloud-bigquery-storage', '2.24.0'),
    ('bigframes', 'bigframes'),
    ('pandas_gbq', 'pandas-gbq'),
    ('tensorflow_io', '--no-deps tensorflow-io')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [4]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

---
## Setup

inputs:

In [18]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [19]:
# source data
BQ_PROJECT = 'bigquery-public-data'
BQ_DATASET = 'ml_datasets'
BQ_TABLE = 'ulb_fraud_detection'

packages:

In [20]:
import timeit
import concurrent.futures

import pandas as pd
from google.cloud import bigquery
from google.cloud import bigquery_storage
import bigframes.pandas as bpd

clients:

In [21]:
bq = bigquery.Client(project = PROJECT_ID)
bqstorage = bigquery_storage.BigQueryReadClient()
bpd.options.bigquery.project = PROJECT_ID

parameters:

In [22]:
BQ_SOURCE = f'{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}'

---
## BigQuery From Jupyter

### BigQuery Cell Magic

See the documentation for [IPython Magics for BigQuery](https://cloud.google.com/python/docs/reference/bigquery/latest/magics)

In [23]:
%%bigquery
SELECT *
FROM bigquery-public-data.ml_datasets.ulb_fraud_detection # this cannot be parameterized with magics
LIMIT 5

Query is running:   0%|          |

Downloading:   0%|          |

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,8748.0,-1.070416,0.304517,2.777064,2.154061,0.254450,-0.448529,-0.398691,0.144672,1.070900,...,-0.122032,-0.182351,0.019576,0.626023,-0.018518,-0.263291,-0.198600,0.098435,0.0,0
1,27074.0,1.165628,0.423671,0.887635,2.740163,-0.338578,-0.142846,-0.055628,-0.015325,-0.213621,...,-0.081184,-0.025694,-0.076609,0.414687,0.631032,0.077322,0.010182,0.019912,0.0,0
2,28292.0,1.050879,0.053408,1.364590,2.666158,-0.378636,1.382032,-0.766202,0.486126,0.152611,...,0.083467,0.624424,-0.157228,-0.240411,0.573061,0.244090,0.063834,0.010981,0.0,0
3,28488.0,1.070316,0.079499,1.471856,2.863786,-0.637887,0.858159,-0.687478,0.344146,0.459561,...,0.048067,0.534713,-0.098645,0.129272,0.543737,0.242724,0.065070,0.023500,0.0,0
4,31392.0,-3.680953,-4.183581,2.642743,4.263802,4.643286,-0.225053,-3.733637,1.273037,0.015661,...,0.649051,1.054124,0.795528,-0.901314,-0.425524,0.511675,0.125419,0.243671,0.0,0


---
## BigQuery From Python With BigFrames API

Use the [BigQuery BigFrames API](https://cloud.google.com/python/docs/reference/bigframes/latest) to query BigQuery and directly interact with BigQuery objects.

In [58]:
bq_data_bigframes = bpd.read_gbq(f'{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')
bq_data_bigframes.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,72890.0,-1.22258,-0.017622,2.317581,-1.547722,-0.958068,-0.370571,-0.583838,0.384328,-0.72238,...,0.430025,1.217131,-0.463494,0.456253,0.385304,-0.104713,-0.303068,-0.300302,5.9,0
1,131206.0,1.967597,-1.009301,-1.970656,-0.406056,1.614598,3.92548,-1.209586,0.952736,-0.429297,...,-0.288566,-0.420307,0.258054,0.632264,-0.148758,-0.656398,0.077885,-0.027551,59.0,0
2,122831.0,2.290614,-1.288035,-1.091499,-1.591945,-0.983697,-0.58711,-0.952236,-0.272064,-1.392405,...,-0.161871,0.04143,0.225622,0.672485,-0.105101,-0.194599,0.007488,-0.040725,30.0,0
3,68397.0,1.258859,0.440981,0.331167,0.681581,-0.267935,-1.046229,0.163925,-0.269223,-0.142249,...,-0.27486,-0.734847,0.116306,0.376938,0.25547,0.090629,-0.015355,0.033149,0.89,0
4,152137.0,2.023988,-0.351874,-0.494781,0.36047,-0.400929,-0.202362,-0.544039,-0.078031,1.364484,...,0.160192,0.774027,0.021697,-0.601828,0.029147,-0.175735,0.04743,-0.041086,9.99,0


In [59]:
type(bq_data_bigframes)

bigframes.dataframe.DataFrame

In [60]:
bq_data_bigframes.shape

(284807, 31)

In [61]:
bq_data_bigframes = bq_data_bigframes.head().to_pandas()
type(bq_data_bigframes)

pandas.core.frame.DataFrame

In [62]:
bq_data_bigframes.shape

(5, 31)

---
## BigQuery From Python

Use the [BigQuery Python Client](https://cloud.google.com/python/docs/reference/bigquery/latest) to query BigQuery and return results as Pandas dataframes.

### BigQuery API From Python


In [24]:
query = f"""
    SELECT * 
    FROM `{BQ_SOURCE}`
    LIMIT 5
"""
preview = bq.query(query = query).to_dataframe()
preview

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,8748.0,-1.070416,0.304517,2.777064,2.154061,0.254450,-0.448529,-0.398691,0.144672,1.070900,...,-0.122032,-0.182351,0.019576,0.626023,-0.018518,-0.263291,-0.198600,0.098435,0.0,0
1,27074.0,1.165628,0.423671,0.887635,2.740163,-0.338578,-0.142846,-0.055628,-0.015325,-0.213621,...,-0.081184,-0.025694,-0.076609,0.414687,0.631032,0.077322,0.010182,0.019912,0.0,0
2,28292.0,1.050879,0.053408,1.364590,2.666158,-0.378636,1.382032,-0.766202,0.486126,0.152611,...,0.083467,0.624424,-0.157228,-0.240411,0.573061,0.244090,0.063834,0.010981,0.0,0
3,28488.0,1.070316,0.079499,1.471856,2.863786,-0.637887,0.858159,-0.687478,0.344146,0.459561,...,0.048067,0.534713,-0.098645,0.129272,0.543737,0.242724,0.065070,0.023500,0.0,0
4,31392.0,-3.680953,-4.183581,2.642743,4.263802,4.643286,-0.225053,-3.733637,1.273037,0.015661,...,0.649051,1.054124,0.795528,-0.901314,-0.425524,0.511675,0.125419,0.243671,0.0,0


### BigQuery Python Client: Helper Function

In [25]:
def bq_runner(query):
    return bq.query(query = query)

In [26]:
bq_runner(
    query = f"""
        SELECT * 
        FROM `{BQ_SOURCE}`
        LIMIT 5
    """
).to_dataframe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,8748.0,-1.070416,0.304517,2.777064,2.154061,0.254450,-0.448529,-0.398691,0.144672,1.070900,...,-0.122032,-0.182351,0.019576,0.626023,-0.018518,-0.263291,-0.198600,0.098435,0.0,0
1,27074.0,1.165628,0.423671,0.887635,2.740163,-0.338578,-0.142846,-0.055628,-0.015325,-0.213621,...,-0.081184,-0.025694,-0.076609,0.414687,0.631032,0.077322,0.010182,0.019912,0.0,0
2,28292.0,1.050879,0.053408,1.364590,2.666158,-0.378636,1.382032,-0.766202,0.486126,0.152611,...,0.083467,0.624424,-0.157228,-0.240411,0.573061,0.244090,0.063834,0.010981,0.0,0
3,28488.0,1.070316,0.079499,1.471856,2.863786,-0.637887,0.858159,-0.687478,0.344146,0.459561,...,0.048067,0.534713,-0.098645,0.129272,0.543737,0.242724,0.065070,0.023500,0.0,0
4,31392.0,-3.680953,-4.183581,2.642743,4.263802,4.643286,-0.225053,-3.733637,1.273037,0.015661,...,0.649051,1.054124,0.795528,-0.901314,-0.425524,0.511675,0.125419,0.243671,0.0,0


### BigQuery Python Client: Using Query Job Attributes and Methods

Query Jobs have Methods and Attributes that can benefit the Python workflow:
- Query Job [Methods](https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.job.QueryJob.html#google.cloud.bigquery.job.QueryJob:~:text=for%20accurate%20signature.-,Methods,-__init__(job_id%2C%C2%A0query)
- Query Job [Attributes](https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.job.QueryJob.html#google.cloud.bigquery.job.QueryJob:~:text=from%20a%20QueryJob-,Attributes,-allow_large_results)

BigQuery Query Job (using helper function):

In [27]:
job = bq_runner(
    query = f"""
        SELECT * 
        FROM `{BQ_SOURCE}`
        LIMIT 5
    """
)

Using Query Job Atrributes to get timing:

In [28]:
job.result()
(job.ended-job.started).total_seconds()

0.154

Using Query Job Methods to retrieve result to Pandas dataframe:

In [29]:
job.to_dataframe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,8748.0,-1.070416,0.304517,2.777064,2.154061,0.254450,-0.448529,-0.398691,0.144672,1.070900,...,-0.122032,-0.182351,0.019576,0.626023,-0.018518,-0.263291,-0.198600,0.098435,0.0,0
1,27074.0,1.165628,0.423671,0.887635,2.740163,-0.338578,-0.142846,-0.055628,-0.015325,-0.213621,...,-0.081184,-0.025694,-0.076609,0.414687,0.631032,0.077322,0.010182,0.019912,0.0,0
2,28292.0,1.050879,0.053408,1.364590,2.666158,-0.378636,1.382032,-0.766202,0.486126,0.152611,...,0.083467,0.624424,-0.157228,-0.240411,0.573061,0.244090,0.063834,0.010981,0.0,0
3,28488.0,1.070316,0.079499,1.471856,2.863786,-0.637887,0.858159,-0.687478,0.344146,0.459561,...,0.048067,0.534713,-0.098645,0.129272,0.543737,0.242724,0.065070,0.023500,0.0,0
4,31392.0,-3.680953,-4.183581,2.642743,4.263802,4.643286,-0.225053,-3.733637,1.273037,0.015661,...,0.649051,1.054124,0.795528,-0.901314,-0.425524,0.511675,0.125419,0.243671,0.0,0


### Indirect use with pandas-gbq

When working with [Pandas](https://pandas.pydata.org/docs/user_guide/index.html#user-guide) the methods above show the client returning data to pandas dataframes.  This section will show a pandas mudule, [pandas-gbq](https://pandas-gbq.readthedocs.io/en/latest/) that wraps the BigQuery client so that pandas can retrieve BigQuery data to dataframes.

References:
- [Comparison of BigQuery Client with pandas-gbq](https://cloud.google.com/bigquery/docs/pandas-gbq-migration)

#### Using pandas-gbq

In [30]:
query = f"""
SELECT * 
FROM `{BQ_SOURCE}`
LIMIT 5
"""
df = pd.read_gbq(query, project_id = PROJECT_ID)

In [31]:
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,8748.0,-1.070416,0.304517,2.777064,2.154061,0.254450,-0.448529,-0.398691,0.144672,1.070900,...,-0.122032,-0.182351,0.019576,0.626023,-0.018518,-0.263291,-0.198600,0.098435,0.0,0
1,27074.0,1.165628,0.423671,0.887635,2.740163,-0.338578,-0.142846,-0.055628,-0.015325,-0.213621,...,-0.081184,-0.025694,-0.076609,0.414687,0.631032,0.077322,0.010182,0.019912,0.0,0
2,28292.0,1.050879,0.053408,1.364590,2.666158,-0.378636,1.382032,-0.766202,0.486126,0.152611,...,0.083467,0.624424,-0.157228,-0.240411,0.573061,0.244090,0.063834,0.010981,0.0,0
3,28488.0,1.070316,0.079499,1.471856,2.863786,-0.637887,0.858159,-0.687478,0.344146,0.459561,...,0.048067,0.534713,-0.098645,0.129272,0.543737,0.242724,0.065070,0.023500,0.0,0
4,31392.0,-3.680953,-4.183581,2.642743,4.263802,4.643286,-0.225053,-3.733637,1.273037,0.015661,...,0.649051,1.054124,0.795528,-0.901314,-0.425524,0.511675,0.125419,0.243671,0.0,0


---

## BigQuery Storage API From Python

BigQuery has an API for directly reading and writing to storage - no need for a query which uses the compute side of BigQuery.  This interacts directly with a single table, not a view, and reads/write rows with some basic filterting capabilities to limit the returned records.  Since BigQuery is columnar storage you can boost efficiency by only reading the columns needed with these methods!


### Create A Read Session

Use the [BigQueryReadClient](https://cloud.google.com/python/docs/reference/bigquerystorage/latest/google.cloud.bigquery_storage_v1.client.BigQueryReadClient) to create a read session with [create_read_session](https://cloud.google.com/python/docs/reference/bigquerystorage/latest/google.cloud.bigquery_storage_v1.client.BigQueryReadClient#google_cloud_bigquery_storage_v1_client_BigQueryReadClient_create_read_session).

>`max_stream_count` set the upper limit for the number of streams returned.  The actual number will be determined based on the tables size and storage layout and could be less than the value provided.  Specifying `0` indicates the system can determine the best value for the `max_stream_counts` and uses a default limit of 1000.

The `read_session` is an [bigquery_storage.types.ReadSession()](https://cloud.google.com/python/docs/reference/bigquerystorage/latest/google.cloud.bigquery_storage_v1.types.ReadSession).

Within the`read_session` the `read_options` is an [bigquery_storage.types.ReadSession.TableReadOptions()](https://cloud.google.com/python/docs/reference/bigquerystorage/latest/google.cloud.bigquery_storage_v1.types.ReadSession.TableReadOptions)

In [40]:
read_session = bqstorage.create_read_session(
    request = dict(
        parent = f'projects/{PROJECT_ID}',
        read_session = dict(
            table = f"projects/{BQ_PROJECT}/datasets/{BQ_DATASET}/tables/{BQ_TABLE}",
            data_format = bigquery_storage.types.DataFormat.ARROW,
            read_options = dict(
                #row_restriction = "Amount > 0",
                selected_fields = [f'V{n+1}' for n in range(28)]
            )
        ),
        max_stream_count = 0
    )
)

Check for the number of streams returned.  A value of would mean all records would be read through a single stream.  A value larger than 1 indicates different streams would be used to read different parts of the table.

In [41]:
len(read_session.streams)

2

### Read the table: One Stream At A Time



In [42]:
def read_stream(stream):
    # setup a reader
    reader = bqstorage.read_rows(
        name = stream.name
    )

    # start timer
    start = timeit.default_timer()

    # read rows from reader into a dataframe.  Note this is actually multiple operations - read and convert
    train = reader.to_dataframe()#.drop(['splits', 'transaction_id'], axis = 1)

    # stop timer and calculate elabsed time
    execution_time = timeit.default_timer() - start

    # report data shape and elapsed time
    print(f'The stream ({stream.name}) read {train.shape[0]} rows and {train.shape[1]} columns in {execution_time} seconds')

    return train

In [43]:
train0 = read_stream(read_session.streams[0])

The stream (projects/statmike-mlops-349915/locations/us/sessions/CAISDFlUNnNySnJkeERUehoCamQaAmpxGgJpchoCb2oaAmpzGgJuYRoCb3caAnB6GgJpdxoCaWMaAmpyGgJqYxoCamYaAnBsGgJvdhoCamkaAmlhGgJweRoCb3MaAnB4/streams/GgJqZBoCanEaAmlyGgJvahoCanMaAm5hGgJvdxoCcHoaAml3GgJpYxoCanIaAmpjGgJqZhoCcGwaAm92GgJqaRoCaWEaAnB5GgJvcxoCcHgoAg) read 142736 rows and 28 columns in 0.33875093500046205 seconds


In [44]:
train1 = read_stream(read_session.streams[1])

The stream (projects/statmike-mlops-349915/locations/us/sessions/CAISDFlUNnNySnJkeERUehoCamQaAmpxGgJpchoCb2oaAmpzGgJuYRoCb3caAnB6GgJpdxoCaWMaAmpyGgJqYxoCamYaAnBsGgJvdhoCamkaAmlhGgJweRoCb3MaAnB4/streams/CAEaAmpkGgJqcRoCaXIaAm9qGgJqcxoCbmEaAm93GgJwehoCaXcaAmljGgJqchoCamMaAmpmGgJwbBoCb3YaAmppGgJpYRoCcHkaAm9zGgJweCgC) read 142071 rows and 28 columns in 0.1601793429999816 seconds


In [45]:
train0.shape[0] + train1.shape[0]

284807

In [46]:
train = pd.concat([train0, train1])

In [47]:
train.shape

(284807, 28)

### Read the table: Async Streams

In [48]:
start = timeit.default_timer()

train = []
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    futures = {
        executor.submit(read_stream, stream): stream for stream in read_session.streams
    }
    for future in concurrent.futures.as_completed(futures):
        stream = futures[future]
        train.append(future.result())
        
print(f'The total elapsed time was: {timeit.default_timer() - start}')

The stream (projects/statmike-mlops-349915/locations/us/sessions/CAISDFlUNnNySnJkeERUehoCamQaAmpxGgJpchoCb2oaAmpzGgJuYRoCb3caAnB6GgJpdxoCaWMaAmpyGgJqYxoCamYaAnBsGgJvdhoCamkaAmlhGgJweRoCb3MaAnB4/streams/GgJqZBoCanEaAmlyGgJvahoCanMaAm5hGgJvdxoCcHoaAml3GgJpYxoCanIaAmpjGgJqZhoCcGwaAm92GgJqaRoCaWEaAnB5GgJvcxoCcHgoAg) read 142736 rows and 28 columns in 0.34685096599969256 seconds
The stream (projects/statmike-mlops-349915/locations/us/sessions/CAISDFlUNnNySnJkeERUehoCamQaAmpxGgJpchoCb2oaAmpzGgJuYRoCb3caAnB6GgJpdxoCaWMaAmpyGgJqYxoCamYaAnBsGgJvdhoCamkaAmlhGgJweRoCb3MaAnB4/streams/CAEaAmpkGgJqcRoCaXIaAm9qGgJqcxoCbmEaAm93GgJwehoCaXcaAmljGgJqchoCamMaAmpmGgJwbBoCb3YaAmppGgJpYRoCcHkaAm9zGgJweCgC) read 142071 rows and 28 columns in 0.3689056610000989 seconds
The total elapsed time was: 0.5045286029999261


In [49]:
len(train)

2

In [50]:
train[0].shape, train[1].shape

((142736, 28), (142071, 28))

In [51]:
train = pd.concat(train)
train.shape

(284807, 28)

### Full Example With Bigger Table

In [52]:
read_session = bqstorage.create_read_session(
    request = dict(
        parent = f'projects/{PROJECT_ID}',
        read_session = dict(
            table = f"projects/{BQ_PROJECT}/datasets/google_trends/tables/top_terms",
            data_format = bigquery_storage.types.DataFormat.ARROW,
            read_options = dict(
                #row_restriction = "rank > 10",
                selected_fields = ['dma_id', 'term', 'week', 'score']
            )
        ),
        max_stream_count = 2
    )
)

In [53]:
len(read_session.streams)

2

In [54]:
start = timeit.default_timer()

train = []
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    futures = {
        executor.submit(read_stream, stream): stream for stream in read_session.streams
    }
    for future in concurrent.futures.as_completed(futures):
        stream = futures[future]
        train.append(future.result())
        
print(f'The total elapsed time was: {timeit.default_timer() - start}')

The stream (projects/statmike-mlops-349915/locations/us/sessions/CAISDGxReG1zUTdnRkpmTxoCamQaAmpxGgJpchoCb2oaAmpzGgJuYRoCb3caAnB6GgJpdxoCaWMaAmpyGgJqYxoCamYaAnBsGgJvdhoCamkaAmlhGgJweRoCb3MaAnB4/streams/GgJqZBoCanEaAmlyGgJvahoCanMaAm5hGgJvdxoCcHoaAml3GgJpYxoCanIaAmpjGgJqZhoCcGwaAm92GgJqaRoCaWEaAnB5GgJvcxoCcHgoAg) read 21186824 rows and 4 columns in 5.984298192000097 seconds
The stream (projects/statmike-mlops-349915/locations/us/sessions/CAISDGxReG1zUTdnRkpmTxoCamQaAmpxGgJpchoCb2oaAmpzGgJuYRoCb3caAnB6GgJpdxoCaWMaAmpyGgJqYxoCamYaAnBsGgJvdhoCamkaAmlhGgJweRoCb3MaAnB4/streams/CAEaAmpkGgJqcRoCaXIaAm9qGgJqcxoCbmEaAm93GgJwehoCaXcaAmljGgJqchoCamMaAmpmGgJwbBoCb3YaAmppGgJpYRoCcHkaAm9zGgJweCgC) read 22535176 rows and 4 columns in 7.12685243299984 seconds
The total elapsed time was: 7.403345630000331


In [55]:
len(train)

2

In [56]:
train[0].shape, train[1].shape

((21186824, 4), (22535176, 4))

In [57]:
train = pd.concat(train)
train.shape

(43722000, 4)